## Load_Tokenization

In [6]:
import pandas as pd
import numpy as np
import re
import pickle

In [2]:
import warnings
warnings.filterwarnings(action='ignore')

In [4]:
df = pd.read_csv('./data/ssodam2021_tokenized.csv',encoding='utf-8')

In [18]:
df.iloc[0]

num                                                                    1082901
board                                                                      익게2
title                                                                    해피뉴이어
text                                                      2021년은 모두 행복한 한해가 되길
writer                                                                      익명
upload_date                                                         2021/01/01
upload_time                                                              00:00
view                                                                    1343.0
likes                                                                    195.0
dislikes                                                                   0.0
comments                     앗, 성지다, 첫글 ㅊㅊ, 추천 준다, 내 24살은 인생에서 가장 빛나는 날로 만들...
comments_writer                                                            NaN
comments_cnt                                        

### Preprocessing

In [19]:
def get_noun_list_from_str(msg):
    try :
        return [tuple([re.sub("'",'',y) for y in re.findall('\'.*?\'',x)]) for x in re.findall('\(.*?\)',msg)]
    except :
        return np.NaN

In [20]:
df = df.fillna('')
df.text_tokenized = df.text_tokenized.apply(lambda x: get_noun_list_from_str(x))
df.title_tokenized = df.title_tokenized.apply(lambda x: get_noun_list_from_str(x))
df.comments_tokenized = df.comments_tokenized.apply(lambda x: get_noun_list_from_str(x))
df.comments_writer_tokenized = df.comments_writer_tokenized.apply(lambda x: get_noun_list_from_str(x))
df.upload_date = pd.to_datetime(df.upload_date)

In [82]:
df.head()

,num,board,title,text,writer,upload_date,upload_time,view,likes,dislikes,comments,comments_writer,comments_cnt,text_tokenized,title_tokenized,comments_tokenized,comments_writer_tokenized
0,1082901,익게2,해피뉴이어,2021년은 모두 행복한 한해가 되길,익명,2021-01-01,00:00,1343.0,195.0,0.0,"앗, 성지다, 첫글 ㅊㅊ, 추천 준다, 내 24살은 인생에서 가장 빛나는 날로 만들...",,20.0,"[(2021년, Number), (은, Foreign), (모두, Noun), (행...","[(해피뉴이어, Noun)]","[(앗, Noun), (,, Punctuation), (성지, Noun), (다, ...",[]
1,1082902,익게2,2021년 새해복 많이받으세요,모든 일이 잘 되기를 12시 땡,익명,2021-01-01,00:00,107.0,3.0,0.0,늦었네,"2빠다 ㅎㅎ, 내년에 도전한다",3.0,"[(모든, Noun), (일이, Modifier), (잘, Verb), (되, Ve...","[(2021년, Number), (새해, Noun), (복, Noun), (많이, ...","[(늦었네, Verb)]","[(2, Number), (빠다, Noun), (ㅎㅎ, KoreanParticle)..."
2,1082903,익게2,첫글은 내꼬,예비회계사 나다미,익명,2021-01-01,00:00,139.0,1.0,0.0,"ㄹㅇ 1등이네 ㅋㅋ, 실패",ㅜㅜ,3.0,"[(예비, Noun), (회계사, Noun), (나, Noun), (다미, Noun)]","[(첫, Modifier), (글, Noun), (은, Josa), (내꼬, Noun)]","[(ㄹㅇ, KoreanParticle), (1등, Number), (이네, Fore...","[(ㅜㅜ, KoreanParticle)]"
3,1082905,익게2,땡ㅎㅎㅎㅎㅎㅎㅎ,1등,익명,2021-01-01,00:00,39.0,0.0,0.0,,,0.0,"[(1등, Number)]","[(땡, Noun), (ㅎㅎㅎㅎㅎㅎㅎ, KoreanParticle)]",[],[]
4,1082906,익게2,어디 카운트 다운 하는 곳 없냐,언제바껴,익명,2021-01-01,00:00,84.0,0.0,0.0,232번 불교방송,,1.0,"[(언, Modifier), (제바, Noun), (껴, Verb)]","[(어디, Noun), (카운트, Noun), (다운, Noun), (하는, Ver...","[(232, Number), (번, Noun), (불교, Noun), (방송, No...",[]


In [7]:
with open('./data/ssodam2021_tokenized_df.pkl','rb') as f:
    df = pickle.load(f)

## Word2Vec

In [133]:
#불용어 load
with open('./data/stopwords.txt',encoding='utf-8') as f:
    stopwords = f.read().split()

### 각 단위 별 corpus

In [150]:
#Noun만 선택
corpus = []
for column in ['title_tokenized','text_tokenized','comments_tokenized','comments_writer_tokenized']:
    for text in df[column]:
        _corpus = []
        for word in text:
            if word[1] in ['Noun']:
                if word[0] not in stopwords:
                    _corpus.append(word[0])
        corpus.append(_corpus)

In [8]:
import pickle
with open('./data/nouns_tokenized.pkl','rb') as f:
    corpus = pickle.load(f)

In [217]:
from gensim.models import Word2Vec

model = Word2Vec(sentences=corpus, vector_size=30, window=8, min_count=5, workers=4, sg=0)

In [218]:
model.wv.most_similar('떡밥')

[('인물', 0.7659158706665039),
 ('뇌절', 0.7550745010375977),
 ('인기', 0.7531453967094421),
 ('밈', 0.7330520153045654),
 ('티아라', 0.7312980890274048),
 ('유머', 0.7256724238395691),
 ('서신애', 0.7238665223121643),
 ('팬픽', 0.7227638363838196),
 ('박혜수', 0.7208364009857178),
 ('수진', 0.7172836661338806)]

In [219]:
model = Word2Vec(sentences=corpus, vector_size=100, window=8, min_count=5, workers=4, sg=0)

In [220]:
model.wv.most_similar('떡밥')

[('팬픽', 0.7107526659965515),
 ('의외', 0.702444851398468),
 ('에이프릴', 0.6986013650894165),
 ('뇌절', 0.6946472525596619),
 ('연예', 0.692428708076477),
 ('학폭', 0.6916701197624207),
 ('인물', 0.6886943578720093),
 ('밈', 0.6714619994163513),
 ('연예인', 0.6692429184913635),
 ('저격', 0.66791832447052)]

### 게시글별 corpus

In [239]:
df['total_tokens'] = df.apply(lambda x : x.text_tokenized + x.comments_writer_tokenized + x.comments_tokenized + x.title_tokenized, axis=1)

In [240]:
df.head()

,num,board,title,text,writer,upload_date,upload_time,view,likes,dislikes,comments,comments_writer,comments_cnt,text_tokenized,title_tokenized,comments_tokenized,comments_writer_tokenized,total_tokens
0,1082901,익게2,해피뉴이어,2021년은 모두 행복한 한해가 되길,익명,2021/01/01,00:00,1343.0,195.0,0.0,"앗, 성지다, 첫글 ㅊㅊ, 추천 준다, 내 24살은 인생에서 가장 빛나는 날로 만들...",,20.0,"[(2021년, Number), (은, Foreign), (모두, Noun), (행...","[(해피뉴이어, Noun)]","[(앗, Noun), (,, Punctuation), (성지, Noun), (다, ...",[],"[(2021년, Number), (은, Foreign), (모두, Noun), (행..."
1,1082902,익게2,2021년 새해복 많이받으세요,모든 일이 잘 되기를 12시 땡,익명,2021/01/01,00:00,107.0,3.0,0.0,늦었네,"2빠다 ㅎㅎ, 내년에 도전한다",3.0,"[(모든, Noun), (일이, Modifier), (잘, Verb), (되, Ve...","[(2021년, Number), (새해, Noun), (복, Noun), (많이, ...","[(늦었네, Verb)]","[(2, Number), (빠다, Noun), (ㅎㅎ, KoreanParticle)...","[(모든, Noun), (일이, Modifier), (잘, Verb), (되, Ve..."
2,1082903,익게2,첫글은 내꼬,예비회계사 나다미,익명,2021/01/01,00:00,139.0,1.0,0.0,"ㄹㅇ 1등이네 ㅋㅋ, 실패",ㅜㅜ,3.0,"[(예비, Noun), (회계사, Noun), (나, Noun), (다미, Noun)]","[(첫, Modifier), (글, Noun), (은, Josa), (내꼬, Noun)]","[(ㄹㅇ, KoreanParticle), (1등, Number), (이네, Fore...","[(ㅜㅜ, KoreanParticle)]","[(예비, Noun), (회계사, Noun), (나, Noun), (다미, Noun..."
3,1082905,익게2,땡ㅎㅎㅎㅎㅎㅎㅎ,1등,익명,2021/01/01,00:00,39.0,0.0,0.0,,,0.0,"[(1등, Number)]","[(땡, Noun), (ㅎㅎㅎㅎㅎㅎㅎ, KoreanParticle)]",[],[],"[(1등, Number), (땡, Noun), (ㅎㅎㅎㅎㅎㅎㅎ, KoreanPart..."
4,1082906,익게2,어디 카운트 다운 하는 곳 없냐,언제바껴,익명,2021/01/01,00:00,84.0,0.0,0.0,232번 불교방송,,1.0,"[(언, Modifier), (제바, Noun), (껴, Verb)]","[(어디, Noun), (카운트, Noun), (다운, Noun), (하는, Ver...","[(232, Number), (번, Noun), (불교, Noun), (방송, No...",[],"[(언, Modifier), (제바, Noun), (껴, Verb), (232, N..."


In [241]:
#Noun만 선택
corpus = []
for column in ['total_tokens']:
    for text in df[column]:
        _corpus = []
        for word in text:
            if word[1] in ['Noun']:
                if word[0] not in stopwords:
                    _corpus.append(word[0])
        corpus.append(_corpus)

In [317]:
from gensim.models import Word2Vec

model = Word2Vec(sentences=corpus, vector_size=3, window=3, min_count=10, workers=4, sg=0)

In [326]:
model.wv.most_similar('스쿼트')

[('볶음', 0.9999963045120239),
 ('로제', 0.9999130964279175),
 ('튀김', 0.9998930096626282),
 ('탕', 0.9998358488082886),
 ('국물', 0.9998195767402649),
 ('공원', 0.999745786190033),
 ('가방', 0.9997155070304871),
 ('과일', 0.9996588826179504),
 ('창문', 0.9996466636657715),
 ('빨래', 0.9996120929718018)]

In [315]:
pd.DataFrame(model.wv.vectors).to_csv('./data/word2vectors.tsv',sep='\t', index=False)
pd.DataFrame(model.wv.index_to_key).to_csv('./data/index2key.tsv',sep='\t', index=False)

## BoW

In [163]:
#불용어 load
with open('./data/stopwords.txt',encoding='utf-8') as f:
    stopwords = f.read().split()

In [370]:
#형태소 체크
limit = 0
for row in df.total_tokens:
    if limit > 10:
        break
    for tup in row:
        if tup[1]=='VerbPrefix':
            print(tup)
            limit += 1

('잘', 'VerbPrefix')
('안', 'VerbPrefix')
('안', 'VerbPrefix')
('못', 'VerbPrefix')
('잘', 'VerbPrefix')
('안', 'VerbPrefix')
('잘', 'VerbPrefix')
('안', 'VerbPrefix')
('안', 'VerbPrefix')
('안', 'VerbPrefix')
('안', 'VerbPrefix')
('잘', 'VerbPrefix')
('안', 'VerbPrefix')


### 익게2 데이터로부터 BoW 생성

In [585]:
#Total Token's Bag of Words
corpus = []
for column in ['total_tokens']:
    for text in df.loc[df['board']=='익게2',column]:
        for word in text:
            if word[1]=='Noun':
                if word[0] not in stopwords:
                    corpus.append(word[0])

In [593]:
idx_total = {}
bow_total = []

for token in corpus:
    if token not in idx_total.keys():
        idx_total[token] = len(idx_total)
        bow_total.insert(len(idx_total)-1,1)
    else:
        bow_total[idx_total[token]] += 1
        
idx_key = list(idx_total.keys())

In [10]:
with open('./data/idx_total.pkl','rb') as f:
    idx_total = pickle.load(f)
with open('./data/bow_total.pkl','rb') as f:
    bow_total = pickle.load(f)
idx_key = list(idx_total.keys())

### 익게2의 각 날짜별 BoW 생성

In [596]:
from datetime import datetime, timedelta

#corpus for each day
cfd = pd.DataFrame(columns=['upload_date','BoW'])

for date in date_range(datetime(2021,1,1), datetime(2021,7,1)):
    bow = np.zeros(len(idx_total.keys()))
    
    for token in df.loc[(df.board=='익게2') & (df.upload_date==date),'total_tokens']:
        for word in token:
            if (word[1]=='Noun') and (word[0] not in stopwords):
                try:
                    bow[idx_total[word[0]]] += 1
                except:
                    print('Error occured')
                    
    cfd.loc[len(cfd)] = [date, bow]

In [12]:
with open('./data/cfd.pkl','rb') as f:
    cfd = pickle.load(f)

In [15]:
#BoW 매트릭스
BoW_mat = np.array([list(value) for value in cfd.BoW.values])
BoW_mat

array([[11.,  1., 29., ...,  0.,  0.,  0.],
       [ 0.,  0., 41., ...,  0.,  0.,  0.],
       [ 2.,  3., 42., ...,  0.,  0.,  0.],
       ...,
       [ 3.,  1., 69., ...,  0.,  0.,  0.],
       [ 1.,  3., 53., ...,  0.,  0.,  0.],
       [ 1.,  3., 54., ...,  1.,  2.,  1.]])

### 날짜별 인기 키워드 탐색

#### 지수 정의

급상승 지수
1. 전일 언급 횟수 a
2. 당일 언급된 횟수 b
3. 전체 기간 중 언급 횟수 c
4. 가중치 w0, w1, w2
5. 적당한 상수 K, 아주 작은 상수 E

w0 x log(1+(b - a)/(a + K1)) + w1 x tanh( 0.1 x (b - K2) ) - w2 x c^E

In [16]:
def relu(x):
    if x<0:
        return 0
    else:
        return x

# [w0, w1, w2, K1, K2, E]    
W = [1,1,-0.0001,10,10,1.1]

def _hot_point(a,b,c):
    x0 = np.log(relu(((b-a)/(a+W[3])))+1)
    x1 = np.tanh(0.1*(b-W[4]))
    x2 = pow(c,W[5])
    
    return W[0]*x0, W[1]*x1, W[2]*x2
    
def hot_point(a,b,c):
    x,y,z = _hot_point(a,b,c)
    return x + y + z

def pp2(a,b,idx):
    c = BoW_mat[:,idx].sum()
    x, y, z = _hot_point(a,b,c)
    print(f'상승률지수 : {x}')
    print(f'언급지수   : {y}')
    print(f'패널티     : {z}')
    print(f'최종값     : {x+y+z}')

#### 날짜/단어별 지수 columns 생성

In [266]:
#각 날짜/단어별 급상승 지수 계산
cfd['points'] = [np.zeros(len(bow_total)) for i in range(len(cfd))]

for i in range(1,len(BoW_mat)):
    for j in range(len(bow_total)):
        cfd.points[i][j] = hot_point(BoW_mat[i-1,j],BoW_mat[i,j],BoW_mat[:,j].sum())

In [267]:
#Points 매트릭스
Points = np.array([list(value) for value in cfd.points.values])
Points

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.78060801, -0.77416411, -0.49338631, ..., -0.76169416,
        -0.76180851, -0.76169416],
       [-0.50072907, -0.35457346, -0.74149917, ..., -0.76169416,
        -0.76180851, -0.76169416],
       ...,
       [-0.36101737, -0.63355764, -0.33941053, ..., -0.76169416,
        -0.76180851, -0.76169416],
       [-0.73531173, -0.44988364, -0.7579678 , ..., -0.76169416,
        -0.76180851, -0.76169416],
       [-0.73531173, -0.61693773, -0.74215272, ..., -0.62108769,
        -0.48192957, -0.62108769]])

- __불용어 추가__

In [268]:
X_list = ['데','뭐','게','익','층','실','곤','급','관','쪽','번','중','감','명','점','곳']
for idx in [idx_total[key] for key in X_list]:
    Points[:,idx] = 0

#### 일별 상위 10, 20개 키워드 추려내기

In [269]:
#index 찾아 True로 저장할 ndarray 정의
top10s = np.zeros(Points.shape, dtype=bool)
top20s = np.zeros(Points.shape, dtype=bool)

for i in range(1,len(Points)):
    #날짜별 점수 상위 10개만 추림
    limit_10 = np.sort(Points[i,:])[-10]
    limit_20 = np.sort(Points[i,:])[-20]
    for j in range(Points.shape[1]):
        if Points[i,j]>=limit_20:
            #상위 20개에 대해 True값으로 저장
            top20s[i,j] = True
            
            if Points[i,j]>=limit_10:
                #상위 10개에 대해 True값으로 저장
                top10s[i,j] = True

top10s

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [270]:
#일별 상위 10개 키워드 저장
top10_keywords = [[idx_key[int(j)] for j in np.arange(top10s.shape[1])[top10s[i]]] for i in range(len(top10s))]
#일별 상위 20개 키워드 저장
top20_keywords = [[idx_key[int(j)] for j in np.arange(top20s.shape[1])[top20s[i]]] for i in range(len(top20s))]

In [271]:
top20_keywords[1]

['선택',
 '얘',
 '배려',
 '경험',
 '룸메',
 '족보',
 '생리',
 '대학원',
 '서로',
 '헬스장',
 '밑',
 '약',
 '조언',
 '중대',
 '연구',
 '김치',
 '냉동',
 '감염',
 '모닝콜',
 '쓰레기']

#### 상위 10개 키워드 중 빈출 키워드 찾기

__top10s__의 __BoW__ 생성

In [272]:
top10s_idx = {}
top10s_bow = []

for day_keywords in top10_keywords:
    for keyword in day_keywords:
        if keyword not in top10s_idx.keys():
            top10s_idx[keyword]=len(top10s_idx)
            top10s_bow.insert(len(top10s_idx)-1,1)
        else :
            top10s_bow[top10s_idx[keyword]]+=1
            
top10s_bow = np.array(top10s_bow)

__BoW__를 __DataFrame__화

In [273]:
top10s_keys = [key for key, value in top10s_idx.items()]

df_10bow = pd.DataFrame(np.array([top10s_keys,top10s_bow]).T,
                      columns = ['keyword','count'])
df_10bow['count'] = df_10bow['count'].astype(int)

df_10bow.head()

,keyword,count
0,배려,1
1,룸메,1
2,생리,3
3,헬스장,1
4,조언,1


In [274]:
df_10bow.sort_values('count',ascending=False)[:50]

,keyword,count
114,연애,8
420,외모,8
17,변호사,8
69,문과,8
577,아빠,7
33,일본,7
223,의사,7
65,의대,6
294,대출,6
64,중국,6


#### 상위 20개 키워드 중 빈출 키워드 찾기

__top20s__의 __BoW__ 생성

In [275]:
top20s_idx = {}
top20s_bow = []

for day_keywords in top20_keywords:
    for keyword in day_keywords:
        if keyword not in top20s_idx.keys():
            top20s_idx[keyword]=len(top20s_idx)
            top20s_bow.insert(len(top20s_idx)-1,1)
        else :
            top20s_bow[top20s_idx[keyword]]+=1
            
top20s_bow = np.array(top20s_bow)

__BoW__를 __DataFrame__화

In [276]:
top20s_keys = [key for key, value in top20s_idx.items()]

df_20bow = pd.DataFrame(np.array([top20s_keys,top20s_bow]).T,
                      columns = ['keyword','count'])
df_20bow['count'] = df_20bow['count'].astype(int)

df_20bow.head()

,keyword,count
0,선택,3
1,얘,1
2,배려,2
3,경험,1
4,룸메,1


In [277]:
df_20bow.sort_values('count',ascending=False)[:50]

,keyword,count
35,변호사,14
178,결혼,13
137,문과,12
40,휴학,12
271,아빠,12
70,외모,12
24,동생,12
220,연애,11
63,미국,11
518,팬,10


__가중치 조정 기록 및 결과 저장__

In [278]:
'''
import pickle
f_idx = '2nd'
with open(f'{f_idx}_weights.pkl','wb') as f:
    pickle.dump(W,f)
with open(f'{f_idx}_cfd.pkl','wb') as f:
    pickle.dump(cfd,f)
with open(f'{f_idx}_10bow.pkl','wb') as f:
    pickle.dump(df_10bow,f)
with open(f'{f_idx}_20bow.pkl','wb') as f:
    pickle.dump(df_20bow,f)
'''

__불러오기__

In [14]:
with open('./data/10bow.pkl','rb') as f:
    df_10bow = pickle.load(f)
with open('./data/20bow.pkl','rb') as f:
    df_20bow = pickle.load(f)

---

## 월별 인기 키워드 찾기

### 작년 12월 데이터를 Load

In [27]:
tokens_20 = pd.read_csv('./data/ssodam2020_all_tokened_final.csv', encoding='utf-8')
#날짜 데이터를 datetime으로.
tokens_20.upload_date = pd.to_datetime(tokens_20.upload_date)
tokens_20.head()

,num,board,title,text,writer,upload_date,upload_time,view,likes,dislikes,comments,comments_writer,comments_cnt,popularity,text_tokenized,title_tokenized,comment_tokenized
0,813032,익게2,2020년 파이팅!!,모두들 파이팅,익명,2020/01/01,00:00,1375,168,0,"['올 첫글 ㅋㅋ', '아 너 뭔데 나보다 일찍 썼냐ㅡㅡ추천이나 먹어라', '첫글 ...",NaN,14,1,"[('파이팅', 'Noun')]","[('2020년', 'Number'), ('파이팅', 'Noun'), ('!!', ...","[(""['"", 'Punctuation'), ('올', 'Verb'), ('첫', '..."
1,813033,익게2,새해,ㅊㅊ,익명,2020/01/01,00:00,97,2,0,NaN,NaN,0,0,"[('ㅊㅊ', 'KoreanParticle')]","[('새해', 'Noun')]",NaN
2,813034,익게2,2020 첫 글 ㄱㅈㅇ,ㅇㅇ,익명,2020/01/01,00:00,128,1,0,NaN,NaN,0,0,"[('ㅇㅇ', 'KoreanParticle')]","[('2020', 'Number'), ('첫', 'Noun'), ('글', 'Nou...",NaN
3,813037,익게2,ㅅㅅ,ㅅㅅ,익명,2020/01/01,00:00,70,0,0,NaN,NaN,0,0,"[('ㅅㅅ', 'KoreanParticle')]","[('ㅅㅅ', 'KoreanParticle')]",NaN
4,813038,익게2,스물넷이다,뭐했다고 벌써,익명,2020/01/01,00:00,124,3,0,"['어? 너두?', '난방금전까지 스물넷..']",NaN,2,0,"[('뭐', 'Noun'), ('했다고', 'Verb'), ('벌써', 'Noun')]","[('스물넷', 'Noun'), ('이다', 'Josa')]","[(""['"", 'Punctuation'), ('?', 'Punctuation'), ..."


In [39]:
def get_noun_list_from_str(msg):
    try :
        return [tuple([re.sub("'",'',y) for y in re.findall('\'.*?\'',x)]) for x in re.findall('\(.*?\)',msg)]
    except :
        return np.NaN

In [69]:
#12월 데이터만 추려내어 처리
tokens_20_12 = tokens_20[tokens_20.upload_date.apply(lambda x: True if x.month==12 else False)]
tokens_20_12 = tokens_20_12.fillna('')
tokens_20_12.text_tokenized = tokens_20_12.text_tokenized.apply(lambda x: get_noun_list_from_str(x))
tokens_20_12.title_tokenized = tokens_20_12.title_tokenized.apply(lambda x: get_noun_list_from_str(x))
tokens_20_12.comment_tokenized = tokens_20_12.comment_tokenized.apply(lambda x: get_noun_list_from_str(x))
tokens_20_12['comments_writer_tokenized'] = [[] for _ in range(len(tokens_20_12))]
tokens_20_12.rename(columns={'comment_tokenized':'comments_tokenized'},inplace=True)
tokens_20_12.head()

,num,board,title,text,writer,upload_date,upload_time,view,likes,dislikes,comments,comments_writer,comments_cnt,popularity,text_tokenized,title_tokenized,comments_tokenized,comments_writer_tokenized
161968,1058725,익게2,엽기붕어빵ㅋㅋ,붕어빵10개 1만 6천원ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ...,익명,2020-12-01,00:06,124,0,0,['ㅈ같은 프리미엄화 좀 하지마라ㅠ'],,1,0,"[(붕어빵, Noun), (10, Number), (개, Noun), (1만, Nu...","[(엽기, Noun), (붕어빵, Noun), (ㅋㅋ, KoreanParticle)]","[("", ,), (ㅈ, KoreanParticle), (같은, Adjective),...",[]
161969,1058727,익게2,다미들 푸우욱 잘 자라옹,졸리다냥,익명,2020-12-01,00:07,143,7,0,,,0,0,"[(졸리다냥, Verb)]","[(푸우, Noun), (욱, Noun), (자라, Noun), (옹, Noun)]",[],[]
161970,1058728,익게2,나 시 써봤는데 어때,거울 속에 한 사내가 서있다사내는 거울 속 자신의 모습이 볼품없다사내는 그런 자신의...,익명,2020-12-01,00:08,245,7,0,['거사사 너때 때멈 살그... 라고요???? 말이 너무 심하시네요!! 비추드립니다...,"['뭐라는거야ㅋㅋㅋㅋ', '저도 쓰고보니 이상 거울이랑 윤동주 자화상이랑 살짝 섞인...",8,0,"[(거울, Noun), (속, Noun), (사내, Noun), (서있다, Verb...","[(시, Noun), (써, Verb), (봤는데, Verb)]","[("", ,), (사사, Noun), (때멈, Noun), (살그, Noun), (...",[]
161971,1058729,익게2,민법 한번 공부해 보고싶은데.. 인강 뭐 들어야할지 모르겠어ㅜ,세무사 민법이랑 법무사 민법...,익명,2020-12-01,00:09,221,0,2,"['로스쿨 민법 어때? - 로스쿨다미', '자기계발용으로는 적합한지 모르겠지만ㅋㅋㅋ...","['헤헤 그거 들어볼까?? 혹시 추천 강사님 물어봐도돼?ㅎ', 'ㅋㅋㅋㅋ']",8,0,"[(세무사, Noun), (민법, Noun), (법무사, Noun), (민법, No...","[(민법, Noun), (한번, Noun), (공부, Noun), (해, Verb)...","[("", ,), (로스쿨, Noun), (민법, Noun), (?, Punctuat...",[]
161972,1058730,익게2,가정내에서도 결국 사회생활하듯이 살아야할까,나는 엄마랑 감정적으로 친해지고 싶지 않아말하는 관점이 너어어어무 다르고 남 험담을...,익명,2020-12-01,00:10,376,11,0,"['솔직히 내가 그 상황이 아니니까 답은 알 수 없지만, 나라면 안 드려 이번에 드...",['진짜 고민이야 ㅠㅠ 정말 오늘 아침도 또 아빠욕하길래 못들어주겟다고 하고 나왔는...,11,0,"[(엄마, Noun), (랑, Josa), (감정, Noun), (적, Suffix...","[(가정, Noun), (내, Noun), (에서도, Josa), (사회생활, No...","[("", ,), (솔직히, Adjective), (내, Noun), (상황, Nou...",[]


In [72]:
tokens_20_12.columns

Index(['num', 'board', 'title', 'text', 'writer', 'upload_date', 'upload_time',
       'view', 'likes', 'dislikes', 'comments', 'comments_writer',
       'comments_cnt', 'popularity', 'text_tokenized', 'title_tokenized',
       'comments_tokenized', 'comments_writer_tokenized'],
      dtype='object')

In [73]:
df.columns

Index(['num', 'board', 'title', 'text', 'writer', 'upload_date', 'upload_time',
       'view', 'likes', 'dislikes', 'comments', 'comments_writer',
       'comments_cnt', 'text_tokenized', 'title_tokenized',
       'comments_tokenized', 'comments_writer_tokenized'],
      dtype='object')

In [84]:
df2 = pd.concat([tokens_20_12,df])

### 익게2 12월~6월간 BoW 생성

전체 데이터프레임을 월별로 groupby.

In [75]:
cfm_dict = dict.fromkeys(['upload_month','title_tokenized','text_tokenized','comments_tokenized','comments_writer_tokenized'])

for key in cfm_dict.keys():
    if key == 'upload_month':
        cfm_dict[key] = [12]+[i+1 for i in range(6)]
    else:
        cfm_dict[key] = [[] for i in range(7)]

cfm_dict

{'upload_month': [12, 1, 2, 3, 4, 5, 6],
 'title_tokenized': [[], [], [], [], [], [], []],
 'text_tokenized': [[], [], [], [], [], [], []],
 'comments_tokenized': [[], [], [], [], [], [], []],
 'comments_writer_tokenized': [[], [], [], [], [], [], []]}

In [658]:
_df = df2.loc[df2.board=='익게2']

for idx in range(len(_df)):
    if (idx+1)%10000==0:
        print('index :',idx,'processing...')
    row = _df.iloc[idx]
    #데이터프레임으로부터 월 인덱스 탐색
    month = 0 if row.upload_date.month==12 else row.upload_date.month
    #칼럼별 해당 인덱스에 해당되는 토큰 저장
    #50단어가 넘는 장문의 글은 제거하고 댓글과 제목만 남김.
    if len(row.text_tokenized) > 50:
        pass
    else :
        cfm_dict['text_tokenized'][month] += row.text_tokenized
    cfm_dict['title_tokenized'][month] += row.title_tokenized
    cfm_dict['comments_tokenized'][month] += row.comments_tokenized
    cfm_dict['comments_writer_tokenized'][month] += row.comments_writer_tokenized

index : 9999 processing...
index : 19999 processing...
index : 29999 processing...
index : 39999 processing...
index : 49999 processing...
index : 59999 processing...
index : 69999 processing...


In [659]:
cfm = pd.DataFrame(cfm_dict)
cfm['total_tokens'] = cfm.title_tokenized + cfm.text_tokenized + cfm.comments_tokenized + cfm.comments_writer_tokenized
cfm

,upload_month,title_tokenized,text_tokenized,comments_tokenized,comments_writer_tokenized,total_tokens
0,12,"[(엽기, Noun), (붕어빵, Noun), (ㅋㅋ, KoreanParticle)...","[(붕어빵, Noun), (10, Number), (개, Noun), (1만, Nu...","[("", ,), (ㅈ, KoreanParticle), (같은, Adjective),...",[],"[(엽기, Noun), (붕어빵, Noun), (ㅋㅋ, KoreanParticle)..."
1,1,"[(해피뉴이어, Noun), (2021년, Number), (새해, Noun), (...","[(2021년, Number), (은, Foreign), (모두, Noun), (행...","[(앗, Noun), (,, Punctuation), (성지, Noun), (다, ...","[(2, Number), (빠다, Noun), (ㅎㅎ, KoreanParticle)...","[(해피뉴이어, Noun), (2021년, Number), (새해, Noun), (..."
2,2,"[(않이, Verb), (모해, Noun), (따고, Verb), (벌써, Noun...","[(2월, Number), (이냐, Foreign), (지인, Noun), (이, ...","[(난, Noun), (마카롱, Noun), (세트, Noun), (했아, Verb...","[(오, Noun), (좋다, Adjective), (고마워, Adjective),...","[(않이, Verb), (모해, Noun), (따고, Verb), (벌써, Noun..."
3,3,"[(약, Noun), (19, Number), (저가, Noun), (코스프레, N...","[(취미, Noun), (로, Josa), (배워, Verb), (보고싶어, Ver...","[(진짜, Noun), (이형, Noun), (꾸준하네, Adjective), (,...","[(고마워, Adjective), (덕분, Noun), (에, Josa), (등업,...","[(약, Noun), (19, Number), (저가, Noun), (코스프레, N..."
4,4,"[(만우절, Noun), (이, Josa), (벌써, Noun), (끝나, Verb...","[(아쉽다, Adjective), (아쉬워, Adjective), (연애, Noun...","[(스타트, Noun), (가, Josa), (좋구만, Adjective), (ㄱㄱ...","[(나, Noun), (에게, Josa), (있다고, Adjective), (,, ...","[(만우절, Noun), (이, Josa), (벌써, Noun), (끝나, Verb..."
5,5,"[(2021년, Number), (5월, Number), (첫, Noun), (글,...","[(새로운, Adjective), (달도, Noun), (희망, Noun), (차게...","[(뭔, Modifier), (데, Noun), (벌써, Noun), (5월, Nu...","[(아, Exclamation), (진짜, Noun), (로, Josa), (그럼,...","[(2021년, Number), (5월, Number), (첫, Noun), (글,..."
6,6,"[(지금, Noun), (퇴근, Noun), (하는, Verb), (길, Noun)...","[(입사, Noun), (1년, Number), (아직, Adverb), (안된, ...","[(화이팅, Noun), (시간, Noun), (이, Josa), (지나면, Ver...","[(전망, Noun), (좋고, Adjective), (과제, Noun), (하기,...","[(지금, Noun), (퇴근, Noun), (하는, Verb), (길, Noun)..."


In [660]:
#Total Token's Bag of Words
corpus_extended = []
for column in ['total_tokens']:
    for text in cfm[column]:
        for word in text:
            try:
                if word[1]=='Noun':
                    if word[0] not in stopwords:
                        corpus_extended.append(word[0])
            except:
                pass

In [661]:
idx_extended = {}
bow_extended = []

for token in corpus_extended:
    if token not in idx_extended.keys():
        idx_extended[token] = len(idx_extended)
        bow_extended.insert(len(idx_extended)-1,1)
    else:
        bow_extended[idx_extended[token]] += 1
        
idx_extended_key = list(idx_extended.keys())

In [664]:
'''
with open('./data/idx_extended.pkl','wb') as f:
    pickle.dump(idx_extended,f)
with open('./data/bow_extended.pkl','wb') as f:
    pickle.dump(bow_extended,f)
'''
with open('./data/idx_extended.pkl','rb') as f:
    idx_extended = pickle.load(f)
with open('./data/bow_extended.pkl','rb') as f:
    bow_extended = pickle.load(f)
idx_extended_key = list(idx_extended.keys())

"\nwith open('./data/idx_extended.pkl','rb') as f:\n    idx_extended = pickle.load(f)\nwith open('./data/bow_extended.pkl','rb') as f:\n    bow_extended = pickle.load(f)\nidx_extended_key = list(idx_extended.keys())\n"

###  Word별 게시글 수 탐색

In [983]:
def return_words(x):
    output = []
    for token in x:
        if len(token)>1:
            output.append(token[0])
        else:
            pass
    return output

df2['total_tokens'] = df2.title_tokenized + df2.text_tokenized + df2.comments_tokenized + df2.comments_writer_tokenized
df2['_total_tokens'] = df2['total_tokens'].apply(lambda x : return_words(x))

In [1554]:
from datetime import datetime
month_filter = []
for i in range(7):
    if i==0:
        month_filter.append((datetime(2020,12,1)<=df2.upload_date) & (df2.upload_date<datetime(2021,1,1)) & (df2.board=='익게2'))
    else:
        month_filter.append((datetime(2021,i,1)<=df2.upload_date) & (df2.upload_date<datetime(2021,i+1,1)) & (df2.board=='익게2'))

In [1016]:
import time
time.time()

1628994322.1614735

In [ ]:
import time

posts_extended = np.zeros((7,len(idx_extended)))

start_time = time.time()

for i, word in enumerate(idx_extended):
    if i%1000 == 0:
        print(i,'번째 단어 처리중...',i-1000,'번째로부터',time.time()-start_time,'초 경과.')
        start_time = time.time()
    _all_posts = df2._total_tokens.apply(lambda x: True if word in x else False)
    for j in range(0,7):
        posts_extended[j,i] = _all_posts[month_filter[j]].sum()

0 번째 단어 처리중... -1000 번째로부터 0.0 초 경과.
1000 번째 단어 처리중... 0 번째로부터 482.98586320877075 초 경과.
2000 번째 단어 처리중... 1000 번째로부터 478.8384139537811 초 경과.
3000 번째 단어 처리중... 2000 번째로부터 480.75723528862 초 경과.
4000 번째 단어 처리중... 3000 번째로부터 477.94149589538574 초 경과.
5000 번째 단어 처리중... 4000 번째로부터 479.8677806854248 초 경과.
6000 번째 단어 처리중... 5000 번째로부터 480.109671831131 초 경과.
7000 번째 단어 처리중... 6000 번째로부터 480.0556654930115 초 경과.
8000 번째 단어 처리중... 7000 번째로부터 471.9213664531708 초 경과.
9000 번째 단어 처리중... 8000 번째로부터 459.60025095939636 초 경과.
10000 번째 단어 처리중... 9000 번째로부터 468.11861205101013 초 경과.
11000 번째 단어 처리중... 10000 번째로부터 465.5619957447052 초 경과.
12000 번째 단어 처리중... 11000 번째로부터 464.84220123291016 초 경과.
13000 번째 단어 처리중... 12000 번째로부터 457.94088196754456 초 경과.
14000 번째 단어 처리중... 13000 번째로부터 461.63939476013184 초 경과.
15000 번째 단어 처리중... 14000 번째로부터 455.12203907966614 초 경과.
16000 번째 단어 처리중... 15000 번째로부터 461.75992250442505 초 경과.
17000 번째 단어 처리중... 16000 번째로부터 457.46056485176086 초 경과.
18000 번째 단어 처리중... 17000 번째로부터 457.92527937

In [ ]:
with open('./data/posts_extended2.pkl','wb') as f:
    pickle.dump(posts_extended,f)

In [ ]:
posts_extended.shape

### 익게2 데이터로부터 월별 BoW 생성

__BoW 생성__

In [665]:
bows = []

for month in range(7):
    bow = np.zeros(len(idx_extended.keys()))
    
    for word in cfm.iloc[month].total_tokens:
        #()등 빈 단어들은 패스
        if len(word)!=2:
            continue
        if (word[1]=='Noun') and (word[0] not in stopwords):
            try:
                bow[idx_extended[word[0]]] += 1
            except:
                print('Error occured')
                
    bows.append(bow)

cfm['BoW'] = bows

In [668]:

with open('./data/cfm.pkl','wb') as f:
    pickle.dump(cfm,f)
'''
with open('./data/cfm.pkl','rb') as f:
    cfm = pickle.load(f)
'''

"\nwith open('./data/cfm.pkl','rb') as f:\n    cfm = pickle.load(f)\n"

### 급상승 지수 계산

idx_total & bow_total로부터 각 월마다 키워드별 급상승지수 계산

월별 급상승 지수
1. 전월 언급 횟수 a
2. 당월 언급된 횟수 b
3. 당월 게시글 작성 횟수 c
4. 전체 기간(2020.12~2021.6) 중 언급 횟수 d
5. 가중치 w0, w1, w2
6. 적당한 상수 K, 아주 작은 상수 E

w0 x log(1+(b - a)/(a + K1)) + w1 x tanh( 0.1 x (b - K2) ) - w2 x c^E

- 월별 등장 횟수는 일별 등장 횟수에 비해 양에 대한 기준치를 높여야 하므로, K1과 K2를 상향 조정

In [1021]:
posts_extended

array([[ 6., 16.,  1., ...,  0.,  0.,  0.],
       [ 4.,  6.,  1., ...,  0.,  0.,  0.],
       [ 2.,  2.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 1.,  5.,  3., ...,  0.,  0.,  0.],
       [ 9.,  6.,  1., ...,  0.,  0.,  0.],
       [ 3.,  6.,  1., ...,  1.,  1.,  1.]])

In [1027]:
BoW_mat

array([[ 36., 194.,   6., ...,   0.,   0.,   0.],
       [ 18.,  29.,   4., ...,   0.,   0.,   0.],
       [  4.,  20.,   0., ...,   0.,   0.,   0.],
       ...,
       [  3.,  38.,   8., ...,   0.,   0.,   0.],
       [ 22.,  18.,   3., ...,   0.,   0.,   0.],
       [ 12.,  39.,   3., ...,   3.,   3.,   3.]])

In [1500]:
def relu(x):
    if x<0:
        return 0
    else:
        return x
    
def sigmoid2(x):
    return 1/( 1+np.exp( -(x-5) ) )

#   [w0, w1, w2,  w3,  K1,  K2,   E]    
W = [1, 0.3, 0.6, -0.05, 10, 100, 1.002]

def _hot_point3(a,b,c,d,e):
    x0 = sigmoid2( ((b-a)/(a+W[4])) + 0.3*((d-c)/(c+W[4]/2)) )
    x1 = np.tanh(0.02*(b-W[5]))
    x2 = np.tanh(0.1*(d-20))-0.5
    x3 = pow(W[6],e) - 1
    
    return W[0]*x0, W[1]*x1, W[2]*x2, W[3]*x3
    
def hot_point3(a,b,c,d,e):
    x,y,z,w = _hot_point3(a,b,c,d,e)
    return x + y + z + w

def pp3(a,b,c,d,idx):
    e = BoW_mat[:,idx].sum()
    x, y, z, w = _hot_point3(a,b,c,d,e)
    print(f'상승률지수 : {x}')
    print(f'언급지수   : {y}')
    print(f'게시글수   : {z}')
    print(f'패널티     : {w}')
    print(f'최종값     : {x+y+z+w}')

In [1501]:
#BoW 매트릭스
BoW_mat = np.array([list(value) for value in cfm.BoW.values])
BoW_mat

array([[ 36., 194.,   6., ...,   0.,   0.,   0.],
       [ 18.,  29.,   4., ...,   0.,   0.,   0.],
       [  4.,  20.,   0., ...,   0.,   0.,   0.],
       ...,
       [  3.,  38.,   8., ...,   0.,   0.,   0.],
       [ 22.,  18.,   3., ...,   0.,   0.,   0.],
       [ 12.,  39.,   3., ...,   3.,   3.,   3.]])

In [1502]:
#각 날짜/단어별 급상승 지수 계산
cfm['points'] = [np.zeros(len(bow_extended)) for i in range(len(cfm))]

for i in range(1,len(BoW_mat)):
    for j in range(len(bow_extended)):
        cfm.points[i][j] = hot_point3(BoW_mat[i-1,j],BoW_mat[i,j],
                                      posts_extended[i-1,j],posts_extended[i,j],
                                      BoW_mat[:,j].sum())

In [1503]:
#Points 매트릭스
Points = np.array([list(value) for value in cfm.points.values])
Points

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-1.13824968, -1.15042475, -1.15872123, ..., -1.16123257,
        -1.16123257, -1.16123257],
       [-1.16295257, -1.19473933, -1.16634607, ..., -1.16123257,
        -1.16123257, -1.16123257],
       ...,
       [-1.16782914, -1.14450623, -1.14367514, ..., -1.16123257,
        -1.16123257, -1.16123257],
       [-1.02467963, -1.15982521, -1.16040464, ..., -1.16123257,
        -1.16123257, -1.16123257],
       [-1.15098264, -1.12397241, -1.15842446, ..., -1.15233796,
        -1.15233796, -1.15233796]])

__추가 불용어 정의__

In [1541]:
#도배된 단어 등 제외
for x_word in ['는','걍','을해','얀','인데','꿀꿀','끝내','쿠','원','쿠나']:
    Points[:,idx_extended[x_word]] = 0.

### 상위 키워드 추리기

In [1542]:
#index 찾아 True로 저장할 ndarray 정의
top10s = np.zeros(Points.shape, dtype=bool)
top20s = np.zeros(Points.shape, dtype=bool)

for i in range(1,len(Points)):
    #날짜별 점수 상위 10개만 추림
    limit_10 = np.sort(Points[i,:])[-10]
    limit_20 = np.sort(Points[i,:])[-20]
    for j in range(Points.shape[1]):
        if Points[i,j]>=limit_20:
            #상위 20개에 대해 True값으로 저장
            top20s[i,j] = True
            
            if Points[i,j]>=limit_10:
                #상위 10개에 대해 True값으로 저장
                top10s[i,j] = True

top10s

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [1543]:
#일별 상위 10개 키워드 저장
top10_keywords = [[idx_extended_key[int(j)] for j in np.arange(top10s.shape[1])[top10s[i]]] for i in range(len(top10s))]
#일별 상위 20개 키워드 저장
top20_keywords = [[idx_extended_key[int(j)] for j in np.arange(top20s.shape[1])[top20s[i]]] for i in range(len(top20s))]

In [1544]:
top10_keywords

[[],
 ['연말정산', '공모전', '눈사람', '능검', '개미', '공공기관', '페스', '페이크', '로토', '팬픽'],
 ['학보', '증원', '쿼터', '뻔뻔', '사인', '뻔선', '연휴', '뻔후', '폭로', '학교폭력'],
 ['피셋', '에이프릴', '결석', '로나', '검찰', '신도시', '투기', '롤린', '단태', '브레이브걸스'],
 ['파상', '벼락치기', '클로즈', '슈퍼리그', '비제', '너비', '애정', '만우절', '서예지', '김정현'],
 ['코난', '참가자', '퇴소', '사물함', '자진', '정민', '퍼즐', '이루리', '공혁준', '또즐'],
 ['부정행위', '텀페', '독일', '사프컴', '관회', '메리트', '광학', '팬덤', '유로', '소희']]

In [1545]:
top20_keywords

[[],
 ['소비자',
  '해커스',
  '연말정산',
  '공모전',
  '수료',
  '눈사람',
  '능검',
  '개미',
  '유의',
  '사지',
  '금리',
  '실적',
  '공공기관',
  '재래시장',
  '페스',
  '페이크',
  '정인',
  '로토',
  '팬픽',
  '강민철'],
 ['학보',
  '증원',
  '물성',
  '졸업식',
  '전공자',
  '분반',
  '쿼터',
  '설날',
  '뻔뻔',
  '실라',
  '사인',
  '괴롭힘',
  '명절',
  '뻔선',
  '연휴',
  '뻔후',
  '폭로',
  '학폭',
  '학교폭력',
  '탈세'],
 ['동선',
  '장관',
  '피셋',
  '에이프릴',
  '결석',
  '벚꽃',
  '수련',
  '로나',
  '검찰',
  '역주행',
  '토지',
  '신도시',
  '서진',
  '등업',
  '투기',
  '롤린',
  '단태',
  '브레이브걸스',
  '마사',
  '하윤'],
 ['파상',
  '벼락치기',
  '테뱅',
  '중도',
  '클로즈',
  '추대',
  '분포',
  '슈퍼리그',
  '시험지',
  '비제',
  '너비',
  '탭',
  '간담',
  '애정',
  '작',
  '권혁',
  '만우절',
  '도지',
  '서예지',
  '김정현'],
 ['코난',
  '범인',
  '전기',
  '참가자',
  '학생회',
  '퇴소',
  '무죄',
  '사물함',
  '실종',
  '자진',
  '파이',
  '어린이',
  '에어컨',
  '정민',
  '퍼즐',
  '이루리',
  '어버이날',
  '공혁준',
  '또즐',
  '니갸르'],
 ['부정행위',
  '경통',
  '열역학',
  '텀페',
  '중회',
  '미적분',
  '떡밥',
  '독일',
  '사프컴',
  '관회',
  '미분',
  '대리',
  '몸매',
  '메리트',
  '광학',
  '팬덤',
  '유로',
  '에고'

__월별 키워드 보기__

In [1546]:
month = 1

[(idx_extended_key[arg],Points[month,arg]) for arg in np.flip(np.argsort(Points[month])) if Points[month,arg]>0.4]

[('눈사람', 1.4388274350075867),
 ('페스', 1.372140387572578),
 ('능검', 1.2968763878230196),
 ('로토', 0.8801678698878485),
 ('페이크', 0.7076259121220817),
 ('팬픽', 0.5900550949301291),
 ('연말정산', 0.5843983551276155),
 ('개미', 0.4655224703512738),
 ('공공기관', 0.45791040726232746),
 ('공모전', 0.448157235581588),
 ('수료', 0.4420610743976951),
 ('금리', 0.4391737496845761),
 ('소비자', 0.4331361420301114),
 ('실적', 0.4265524576852504),
 ('해커스', 0.4249429654020079),
 ('강민철', 0.42325984109543124),
 ('유의', 0.41582487531914836),
 ('사지', 0.41059350647106896),
 ('정인', 0.4054266187954222),
 ('재래시장', 0.40193374821517885)]

### 단어별 검토

In [1548]:
_word = '공공기관'

- 월별 점수

In [1549]:
Points[:,idx_extended[_word]]

array([ 0.        ,  0.45791041, -0.33855227, -0.54824645, -0.93025855,
       -0.17494321, -0.6812255 ])

- 전체 등장 횟수

In [1550]:
bow_extended[idx_extended[_word]]

469

- 월별 등장 횟수

In [1551]:
BoW_mat[:,idx_extended[_word]]

array([ 81., 153.,  62.,  47.,  17.,  68.,  41.])

- 월별 게시글 수

In [1552]:
posts_extended[:,idx_extended[_word]]

array([45., 42., 24., 21., 15., 26., 19.])

- 지수 확인

In [1536]:
_word = '쿠나'

In [1553]:
end = 1
pp3(BoW_mat[end-1,idx_extended[_word]],BoW_mat[end,idx_extended[_word]],
    posts_extended[end-1,idx_extended[_word]], posts_extended[end,idx_extended[_word]],
    idx_extended[_word])

상승률지수 : 0.014389092450465357
언급지수   : 0.23569915770808308
게시글수   : 0.2854458780188709
패널티     : -0.07762372091509183
최종값     : 0.45791040726232746


- 내용 검색 툴

In [951]:
def search_cd(word, x):
    try:
        return True if word in [y[0] for y in x] else False
    except:
        return False

def s_search(word, month, show_urls=True):
    df_month = df2[df2.upload_date.apply(lambda x: True if x.month==month else False)]
    title_cd = df2.title_tokenized.apply(lambda x: search_cd(word,x))
    text_cd = df2.text_tokenized.apply(lambda x: search_cd(word,x))
    comments_cd = df2.comments_tokenized.apply(lambda x: search_cd(word,x))
    comments_writer_cd = df2.comments_writer_tokenized.apply(lambda x: search_cd(word,x))
    df = df_month[title_cd | text_cd | comments_cd | comments_writer_cd]
    if show_urls==True:
        [print('http://www.ssodam.com/content/'+str(number)) for number in df.num]
    return df

def numofposts(word, month):
    df = s_search(word,month,show_urls=False)
    return len(df)

In [1547]:
s_search('공공기관',1)

http://www.ssodam.com/content/1083036
http://www.ssodam.com/content/1083670
http://www.ssodam.com/content/1084556
http://www.ssodam.com/content/1085597
http://www.ssodam.com/content/1086277
http://www.ssodam.com/content/1086384
http://www.ssodam.com/content/1086874
http://www.ssodam.com/content/1087738
http://www.ssodam.com/content/1087918
http://www.ssodam.com/content/1088090
http://www.ssodam.com/content/1088234
http://www.ssodam.com/content/1088256
http://www.ssodam.com/content/1088296
http://www.ssodam.com/content/1089231
http://www.ssodam.com/content/1089673
http://www.ssodam.com/content/1090305
http://www.ssodam.com/content/1090471
http://www.ssodam.com/content/1090747
http://www.ssodam.com/content/1091235
http://www.ssodam.com/content/1091514
http://www.ssodam.com/content/1091547
http://www.ssodam.com/content/1091991
http://www.ssodam.com/content/1092521
http://www.ssodam.com/content/1092998
http://www.ssodam.com/content/1093426
http://www.ssodam.com/content/1093507
http://www.s

,num,board,title,text,writer,upload_date,upload_time,view,likes,dislikes,comments,comments_writer,comments_cnt,popularity,text_tokenized,title_tokenized,comments_tokenized,comments_writer_tokenized,total_tokens,_total_tokens
101,1083036,취업게시판,공공기관 아르바이트와 교내근로 고민입니다,현재 7학기 끝난 학생입니다 제가 사회 생활 경험이 거의 없기도 하고 생활비에도 보...,익명,2021-01-01,02:58,820.0,1.0,0.0,"제가 공공기관 사무보조 진짜 많이 해봤는데, 첫째로는 일을 배우는건 없고 간단한 일...",답변 감사합니다 공부와 경험 중 제가 선택해서 잘 정해야겠군요 혹시 그때 하셨던 기...,18.0,NaN,"[(현재, Noun), (7, Number), (학기, Noun), (끝난, Ver...","[(공공기관, Noun), (아르바이트, Noun), (와, Josa), (교내, ...","[(제, Noun), (가, Josa), (공공기관, Noun), (사무, Noun...","[(답변, Noun), (감사합니다, Verb), (공부, Noun), (와, Jo...","[(공공기관, Noun), (아르바이트, Noun), (와, Josa), (교내, ...","[공공기관, 아르바이트, 와, 교내, 근로, 고민, 입니다, 현재, 7, 학기, 끝..."
550,1083670,취업게시판,수습기간에 코로나 걸리면 짤릴 수도 있나요,맨날 집에만 있을 수도 없고,익명,2021-01-02,09:52,1321.0,4.0,0.0,"해당되면 노동위원회 통해 고소하심 되지 않을까요부당한 해고라고 ㅎㅎ, 정직원들도 놀...","그건 그런데 어차피 그래봤자 회사에서 매장당할듯요ㅎㅎ, 하 진짜 짜증나네요 밖에 가...",17.0,NaN,"[(맨날, Adverb), (집, Noun), (에만, Josa), (있을, Adj...","[(수, Modifier), (습기, Noun), (간, Suffix), (에, J...","[(해당, Noun), (되면, Verb), (노동, Noun), (위원회, Nou...","[(그건, Adverb), (그런데, Conjunction), (어차피, Noun)...","[(수, Modifier), (습기, Noun), (간, Suffix), (에, J...","[수, 습기, 간, 에, 코로나, 걸리면, 짤릴, 수도, 있나요, 맨날, 집, 에만..."
1134,1084556,취업게시판,채용전환인턴 시 네일아트나 피어싱,이번에 운좋게 채용전환인턴으로 근무를 시작하게되었습니다혹시 네일아트 화려하지나 23...,익명,2021-01-03,16:15,1257.0,6.0,0.0,저희 채용인턴은 빨간 네일아트에 개화려한 귀걸이 하던데ㅋㅋㅋㅋㅋ 심지어 본부장실 코...,,18.0,NaN,"[(이번, Noun), (에, Josa), (운, Noun), (좋게, Adject...","[(채용, Noun), (전환, Noun), (인턴, Noun), (시, Noun)...","[(저희, Noun), (채용, Noun), (인턴, Noun), (은, Josa)...",[],"[(채용, Noun), (전환, Noun), (인턴, Noun), (시, Noun)...","[채용, 전환, 인턴, 시, 네일아트, 나, 피어싱, 이번, 에, 운, 좋게, 채용..."
1871,1085597,취업게시판,이번에 서강대학교 장학부서직원채용,공공기관이나 교직원관련 경력없으면,익명,2021-01-04,22:54,1214.0,5.0,0.0,"저도 학부조교 말고는 딱히 접점 없는데 일단 자교버프 믿고 한번 써보려구요ㅋㅋ, 교...",,9.0,NaN,"[(공공기관, Noun), (이나, Josa), (교직원, Noun), (관련, N...","[(이번, Noun), (에, Josa), (서강대, Noun), (학교, Noun...","[(저, Noun), (도, Josa), (학부, Noun), (조교, Noun),...",[],"[(이번, Noun), (에, Josa), (서강대, Noun), (학교, Noun...","[이번, 에, 서강대, 학교, 장학, 부서, 직원, 채용, 공공기관, 이나, 교직원..."
2347,1086277,취업게시판,공기업 취업 희망시 사기업 인턴 영향 문의,"안녕하세요,공사 취준 중 궁금한 점이 있어 글 남깁니다혹시, 공사 입사지원시 사기업...",익명,2021-01-05,23:38,403.0,1.0,0.0,"아니요 문제될거 전혀 없어요, 가끔가다 공공기관 인턴만 서류가점 주는데들도 있긴 한...",답변주신 분들 모두 감사드립니다,5.0,NaN,"[(안녕하세요, Adjective), (,, Punctuation), (공사, No...","[(공기업, Noun), (취업, Noun), (희망, Noun), (시, Noun...","[(아니요, Adjective), (문제, Noun), (될거, Verb), (전혀...","[(답변, Noun), (주신, Noun), (분들, Suffix), (모두, No...","[(공기업, Noun), (취업, Noun), (희망, Noun), (시, Noun...","[공기업, 취업, 희망, 시, 사, 기업, 인턴, 영향, 문의, 안녕하세요, ,, ..."
2412,1086384,익게2,재택근무가 불공정한가,"원래 우리 부서는 재택근무를 권장하는 문화였어공공기관이라 13 재택근무 권장인데, ...",익명,2021-01-06,09:18,306.0,9.0,0.0,"코로나 심한데 에바네, 재택특혜 논리 뭐냐 ㅋㅋ, 전형적인 꼰머마인드, 주어진 일만...","너무 무서워ㅠㅠ 안그래도 회사에서 확진자 한번 나와서 더무섭다, ㅋㅋㅋㅋ근데 뭐라고...",10.0,NaN,"[(원래, Noun), (우리, Noun), (부서, Noun), (는, Josa)...","[(재택근무, Noun), (가, Josa), (불공정, Noun), (한, Jos...","[(코로나, Noun), (심한데, Verb), (에바, Noun), (네, Jos...","[(너무, Adverb), (무서워, Adjective), (ㅠㅠ, KoreanPa...","[(재택근무, Noun), (가, Josa), (불공정, Noun), (한, Jos...","[재택근무, 가, 불공정, 한, 가, 원래, 우리, 부서, 는, 재택근무, 를, 권..."
2750,1086874,익게2,리메이크 웹툰의 저작권에 대한 질문이에요,제가 만약 위대한 개츠비 아니면 데미안 같은 작품을리메이크하거나 웹툰으로 그린다고 ...,익명,2021-01-06,22:33,157.0,1.0,0.0,"서담에서 묻는 건 적절하지 않은 듯로톡이나 지식인 ㄱㄱ, 작가 사후 70년이면 저작...",존버는 승리,5.0,NaN,"[(제, Noun), (가, Josa), (만약, Noun), (위대한, Adjec...","[(리메이크, Noun), (웹툰, Noun), (의, Josa), (저작권, No...","[(서, Modifier), (담, Noun), (에서, Josa), (묻는, Ve...","[(존버, Noun), (는, Josa), (승리, Noun)]","[(리메이크, Noun), (웹툰, Noun), (의, Josa), (저작권, No...","[리메이크, 웹툰, 의, 저작권, 에, 대한, 질문, 이에요, 제, 가, 만약, 위..."
3364,1087738,익게2,다음 시즌에는 얼어붙은 채용시장 좀 풀릴까,"다음시즌에 정부가 공공일자리, 공무원도 제일 많이 뽑는다던데 지원자들이 거기로 좀 ...",익명,2021-01-08,14:05,485.0,5.0,0.0,"누적 취준생들이 많아서 딱히 그렇지는 않을듯, 공공일자리 확대하는게 나라에 장기적으...",그것도 맞는 소리네 좀 오래 취준하신 분들 전문직이나 공직으로 진로 돌려도 워낙 적...,15.0,NaN,"[(다음, Noun), (시즌, Noun), (에, Josa), (정부, Noun)...","[(다음, No

---

,num,board,title,text,writer,upload_date,upload_time,view,likes,dislikes,comments,comments_writer,comments_cnt,text_tokenized,title_tokenized,comments_tokenized,comments_writer_tokenized
0,1082901,익게2,해피뉴이어,2021년은 모두 행복한 한해가 되길,익명,2021-01-01,00:00,1343.0,195.0,0.0,"앗, 성지다, 첫글 ㅊㅊ, 추천 준다, 내 24살은 인생에서 가장 빛나는 날로 만들...",,20.0,"[(2021년, Number), (은, Foreign), (모두, Noun), (행...","[(해피뉴이어, Noun)]","[(앗, Noun), (,, Punctuation), (성지, Noun), (다, ...",[]
1,1082902,익게2,2021년 새해복 많이받으세요,모든 일이 잘 되기를 12시 땡,익명,2021-01-01,00:00,107.0,3.0,0.0,늦었네,"2빠다 ㅎㅎ, 내년에 도전한다",3.0,"[(모든, Noun), (일이, Modifier), (잘, Verb), (되, Ve...","[(2021년, Number), (새해, Noun), (복, Noun), (많이, ...","[(늦었네, Verb)]","[(2, Number), (빠다, Noun), (ㅎㅎ, KoreanParticle)..."
2,1082903,익게2,첫글은 내꼬,예비회계사 나다미,익명,2021-01-01,00:00,139.0,1.0,0.0,"ㄹㅇ 1등이네 ㅋㅋ, 실패",ㅜㅜ,3.0,"[(예비, Noun), (회계사, Noun), (나, Noun), (다미, Noun)]","[(첫, Modifier), (글, Noun), (은, Josa), (내꼬, Noun)]","[(ㄹㅇ, KoreanParticle), (1등, Number), (이네, Fore...","[(ㅜㅜ, KoreanParticle)]"
3,1082905,익게2,땡ㅎㅎㅎㅎㅎㅎㅎ,1등,익명,2021-01-01,00:00,39.0,0.0,0.0,,,0.0,"[(1등, Number)]","[(땡, Noun), (ㅎㅎㅎㅎㅎㅎㅎ, KoreanParticle)]",[],[]
4,1082906,익게2,어디 카운트 다운 하는 곳 없냐,언제바껴,익명,2021-01-01,00:00,84.0,0.0,0.0,232번 불교방송,,1.0,"[(언, Modifier), (제바, Noun), (껴, Verb)]","[(어디, Noun), (카운트, Noun), (다운, Noun), (하는, Ver...","[(232, Number), (번, Noun), (불교, Noun), (방송, No...",[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77975,1200285,익게2,아버지께서 칼럼 쓰셨는데 댓글 한 번씩만 달아줄 수 있을까,2222408544049우리 아버지께서 이번에 짧은 칼럼처럼 블로그에 쓰시는 거 맡...,익명,2021-06-30,23:43,5802.0,374.0,11.0,"비댓으로 달래 너무 귀엽다, 이 글 조회수 늘리려면 제목 바꾸는거 추천 광고글인줄 ...","헉 피드백 반영했어 너무 고마워, 엇 아마 확인은 못하실 텐데 비댓이든 공개댓이든 ...",75.0,"[(2222408544049, Number), (우리, Noun), (아버지, No...","[(아버지, Noun), (께서, Josa), (칼럼, Noun), (쓰셨는데, V...","[(비댓, Noun), (으로, Josa), (달래, Noun), (너무, Adve...","[(헉, Adverb), (피드백, Noun), (반영, Noun), (했어, Ve..."
77976,1200287,익게2,프로포폴 하면 기분이 좋음,하는거 신기하네하면 힙해보여서 하는건가,익명,2021-06-30,23:47,373.0,1.0,1.0,썰로 들은 거라 정확하지는 않는데 지방흡입할 때 고통 줄이려고 프로포폴 투약하다가 ...,,5.0,"[(하는거, Verb), (신기하네하면, Adjective), (힙, Noun), ...","[(프로포폴, Noun), (하면, Verb), (기분, Noun), (이, Jos...","[(썰로, Verb), (들은, Verb), (거, Noun), (라, Josa),...",[]
77977,1200288,익게2,취업 후기 쓰려는데 안올라간다ㅠ,손다쳐서 폰으로 열심히 적었는데나한테왜 이래ㅠ혹시 뭔가 지켜야 하는 양식이 있나요ㅠ...,익명,2021-06-30,23:49,387.0,3.0,0.0,"중간에 임티 넣었었어, 임티, 1 이모티콘 빼기2 사진 많으면 업로드 될 동안 조금...",이건 되네,7.0,"[(손, Noun), (다쳐서, Verb), (폰, Noun), (으로, Josa)...","[(취업, Noun), (후기, Noun), (쓰려는데, Verb), (안, Ver...","[(중간, Noun), (에, Josa), (임티, Noun), (넣었었어, Ver...","[(이건, Noun), (되네, Verb)]"
77978,1200291,익게2,간떨어지는동거 보는 사람,아 이번주 기대했는데 예고편...,익명,2021-06-30,23:55,138.0,6.0,0.0,"너무 오글거리는데 장기용때매 봄 ㅎㅎ, 난 계선우땜에 봐ㅠㅠ, 나도 계선우 때문에 ...",,3.0,"[(아, Exclamation), (이번, Noun), (주, Noun), (기대했...","[(간, Noun), (떨어지는, Verb), (동거, Noun), (보는, Ver...","[(너무, Adverb), (오글거리는데, Verb), (장기, Noun), (용때...",[]


## ETC

In [952]:
numofposts('비건',1)

6

In [953]:
len(idx_extended)

86107

In [954]:
idx_extended

{'엽기': 0,
 '붕어빵': 1,
 '푸우': 2,
 '욱': 3,
 '자라': 4,
 '옹': 5,
 '시': 6,
 '민법': 7,
 '한번': 8,
 '공부': 9,
 '인강': 10,
 '가정': 11,
 '내': 12,
 '사회생활': 13,
 '듯이': 14,
 '살': 15,
 '체지방': 16,
 '수도': 17,
 '요금': 18,
 '납부': 19,
 '기한': 20,
 '계절': 21,
 '수강신청': 22,
 '겨울': 23,
 '어제': 24,
 '오후': 25,
 '필기': 26,
 '추가': 27,
 '합격': 28,
 '오늘': 29,
 '화': 30,
 '면접': 31,
 '간다': 32,
 '챗': 33,
 '이야기': 34,
 '교협': 35,
 '공식': 36,
 '이사장': 37,
 '거절': 38,
 '엽오': 39,
 '너구리': 40,
 '꽃게': 41,
 '라면': 42,
 '노트북': 43,
 '업그레이드': 44,
 '에이스': 45,
 '유문': 46,
 '학년': 47,
 '아씨': 48,
 '발견': 49,
 '코난': 50,
 '전설': 51,
 '최초': 52,
 '사건': 53,
 '태블릿': 54,
 '유저': 55,
 '스캔': 56,
 '선배': 57,
 '조언': 58,
 '코로나': 59,
 '새내기': 60,
 '유예': 61,
 '생': 62,
 '무물': 63,
 '밤새': 64,
 '자바': 65,
 '과제': 66,
 '영어': 67,
 '샤샤': 68,
 '샤': 69,
 '교수': 70,
 '질문': 71,
 '누군가': 72,
 '사실': 73,
 '스마트폰': 74,
 '컴퓨터': 75,
 '맨': 76,
 '씨파': 77,
 '기합': 78,
 '방탄소년단': 79,
 '신곡': 80,
 '빌보드': 81,
 '컴사': 82,
 '막': 83,
 '학기': 84,
 '학점': 85,
 '등록': 86,
 '논개': 87,
 '철인': 88,
 '로욜라': 89,
 '시국'

In [962]:
Posts_mat = np.zeros((7,len(idx_extended)))
for i, key in enumerate(idx_extended):
    print(i)
    for j in range(1,7):
        print(j)
        Posts_mat[j,i] = numofposts(key,i)

0
1
2
3
4
5
6
1
1
2
3
4
5
6
2
1
2
3
4
5
6
3
1
2
3
4
5
6
4
1


KeyboardInterrupt: 

In [ ]:
with open('posts_mat.pkl','wb') as f:
    pickle.dump(Posts_mat,f)

In [958]:
Posts_mat = np.zeros((6,len(idx_extended)))
Posts_mat

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [486]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [737]:
with open('corpus.pkl','wb') as f:
    pickle.dump(corpus,f)
with open('idx_total.pkl','wb') as f:
    pickle.dump(idx_total,f)
with open('bow_total.pkl','wb') as f:
    pickle.dump(bow_total,f)
with open('cfd.pkl','wb') as f:
    pickle.dump(cfd,f)

In [4]:
import pickle

with open('corpus.pkl','rb') as f:
    corpus = pickle.load(f)
with open('idx_total.pkl','rb') as f:
    idx_total = pickle.load(f)
with open('bow_total.pkl','rb') as f:
    bow_total = pickle.load(f)
with open('cfd.pkl','rb') as f:
    cfd = pickle.load(f)

## Topic Modeling

In [650]:
len(df2[df2.num==1096863].text_tokenized.values[0])

1026

In [634]:
len(df.iloc[10912].text_tokenized)

9626

In [635]:
len(df.iloc[10000].text_tokenized)

19

In [611]:
df2[text_cd]

,num,board,title,text,writer,upload_date,upload_time,view,likes,dislikes,comments,comments_writer,comments_cnt,popularity,text_tokenized,title_tokenized,comments_tokenized,comments_writer_tokenized
162331,1059230,익게2,인턴 자소서 쓰니깐 공부시간이 너무 줄어드네 ㅠ,원래 시험 2주전부터 본격적으로 공부하는데 ...,익명,2020-12-01,18:36,211.0,0.0,0.0,"['나도 진짜 이번학기 학점 알파벳 하나씩은 밀린다 ㅂㄷㅂㄷ', '시험공부는 하면 ...",,3.0,0.0,"[(원래, Noun), (시험, Noun), (2, Number), (주전, Nou...","[(인턴, Noun), (자소, Noun), (서, Josa), (쓰니깐, Verb...","[("", ,), (나도, Verb), (학기, Noun), (학점, Noun), (...",[]
163040,1060241,익게2,방호복 입고 수능 치러간 수험생,방호복 입고 수능 치러간 수험생,익명,2020-12-03,10:24,2788.0,75.0,0.0,"['얘 오늘 엄청 기사나온다 ㅋㅋㅋㅋㅋ', '와 답답하고 더워서 집중이 될까...?...",['주섬주섬주섬 '],30.0,1.0,"[(방호복, Noun), (입고, Verb), (수능, Noun), (치러, Ver...","[(방호복, Noun), (입고, Verb), (수능, Noun), (치러, Ver...","[("", ,), (얘, Noun), (오늘, Noun), (엄청, Adverb), ...",[]
163415,1060756,익게2,인기글 간 동생이 수능 포기하고 나온 후기,인기글 간 동생이 수능 포기했다는 다미야저녁에 동생 만나서 밥먹었는데 댓글 써준 다...,익명,2020-12-03,22:23,2403.0,98.0,0.0,['동생 멘탈관리법 필요하다 ... 진짜 지나고보면 고작 수능이라고 생각드는데 벌써...,"['그러게 말이야...', '그랬으면 좋겠다...', '너도 솔직하게 말하는게 좋을...",12.0,1.0,"[(인기, Noun), (글, Noun), (간, Noun), (동생, Noun),...","[(인기, Noun), (글, Noun), (간, Noun), (동생, Noun),...","[("", ,), (동생, Noun), (멘탈, Noun), (관리, Noun), (...",[]
163483,1060854,익게2,다음주부터 기말기간이야?,졸업생 다미인데 오늘 시험 관련 글이 많이 올라오는 것 같아서 갑자기 궁금해지네ㅋㅋ...,익명,2020-12-04,01:21,72.0,0.0,0.0,,,0.0,0.0,"[(졸업생, Noun), (오늘, Noun), (시험, Noun), (관련, Nou...","[(다음주, Noun), (기, Modifier), (말기, Noun), (간, S...",[],[]
163781,1061245,익게2,월가의 영웅 피터 린치가 13년간 운용한 마젤란 펀드 수익률,월가의 영웅이라 불리우는 전설의 펀드매니저 피터 린치가피델리티 책임매니저로서 13년...,익명,2020-12-04,16:57,295.0,5.0,0.0,"['진짜 대단함 그 자체... 주식하면 피터린치 책 정독 필수지', '삭제된 댓글입...",['1600개 종목 관리함 ㅋㅋ'],7.0,0.0,"[(가의, Noun), (영웅, Noun), (이라, Josa), (불리우는, Ve...","[(가의, Noun), (영웅, Noun), (피터, Noun), (린치, Noun...","[("", ,), (대단함, Adjective), (자체, Noun), (..., P...",[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76997,1198774,익게2,얼굴 오른쪽 부분뇌머리통포함이 간헐적으로 저리네,몇달전부터 이러다가 요즘 좀 자주 저려지금도 저리는 느낌이 들어 ...,익명,2021-06-28,23:25,108.0,1.0,0.0,"찍어봐야할듯, 실비보험 빠르게 가입해서 보장받을 수 있을 때 ㄱㄱ, 진지하게 빨리 ...","아휴, 엄마가 나 보험 뭐 들긴했을텐데다미가 말한거랑 다른거려나, 리트 한달도 안...",14.0,NaN,"[(몇, Noun), (달전부터, Verb), (이러다가, Adverb), (요즘,...","[(얼굴, Noun), (오른쪽, Noun), (부분, Noun), (뇌, Noun...","[(찍어, Verb), (봐, Verb), (야할듯, Adjective), (,, ...","[(아, Exclamation), (휴, Exclamation), (,, Punct..."
77006,1198791,익게2,부모님한테 문자로 성적간다는게,정확히 무슨 뜻이야 성적이 ...,익명,2021-06-28,23:33,248.0,1.0,1.0,"후자인거 같음, 서강고자나 고등학교땐 종이 성적표에 싸인받아오라했던거 같은데 ㅋㅋㅋ...","후자라고 고등학교도 안하는짓을 진짜 충격적, 고등학교때도 성적표에 싸인받아가진 않아...",11.0,NaN,"[(정확히, Adjective), (무슨, Noun), (뜻, Noun), (이야,...","[(부모님, Noun), (한테, Josa), (문자, Noun), (로, Josa...","[(후자, Noun), (인거, Josa), (같음, Adjective), (,, ...","[(후자, Noun), (라고, Josa), (고등학교, Noun), (도, Jos..."
77362,1199311,익게2,비행기 도착 예정시간보다 많이 일찍 도착해본사람,ㅇㅅㅇ ...,익명,2021-06-29,19:02,175.0,0.0,0.0,"그럴 일 없다보면 되는데 , 국내선, 서울 제주 기준 아무리 길어도 10분이내, 늦...",ㄴㄴ 외국,8.0,NaN,"[(ㅇㅅㅇ, KoreanParticle), (얼마나, Noun), (일찍, Noun...","[(비행기, Noun), (도착, Noun), (예, Modifier), (정시, ...","[(그럴, Adjective), (일, Noun), (없다, Adjective), ...","[(ㄴㄴ, KoreanParticle), (외국, Noun)]"
77968,1200274,익게2,간동거 도재진 너무좋다,내 이상형,익명,2021-06-30,23:32,56.0,0.0,0.0,"다른 얘기지만 혜리 연기 너무 못해ㅜㅠㅠ 내 몰입 다 깨져, 도재진 나오는 그 웹드...",,2.0,NaN,"[(내, Noun), (이상형, Noun)]","[(간, Noun), (동거, Noun), (도, Noun), (재진, Verb),...","[(다른, Noun), (얘기, Noun), (지만, Josa), (혜리, Noun...",[]


In [590]:
df2

,num,board,title,text,writer,upload_date,upload_time,view,likes,dislikes,comments,comments_writer,comments_cnt,popularity,text_tokenized,title_tokenized,comments_tokenized,comments_writer_tokenized
161968,1058725,익게2,엽기붕어빵ㅋㅋ,붕어빵10개 1만 6천원ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ...,익명,2020-12-01,00:06,124.0,0.0,0.0,['ㅈ같은 프리미엄화 좀 하지마라ㅠ'],,1.0,0.0,"[(붕어빵, Noun), (10, Number), (개, Noun), (1만, Nu...","[(엽기, Noun), (붕어빵, Noun), (ㅋㅋ, KoreanParticle)]","[("", ,), (ㅈ, KoreanParticle), (같은, Adjective),...",[]
161969,1058727,익게2,다미들 푸우욱 잘 자라옹,졸리다냥,익명,2020-12-01,00:07,143.0,7.0,0.0,,,0.0,0.0,"[(졸리다냥, Verb)]","[(푸우, Noun), (욱, Noun), (자라, Noun), (옹, Noun)]",[],[]
161970,1058728,익게2,나 시 써봤는데 어때,거울 속에 한 사내가 서있다사내는 거울 속 자신의 모습이 볼품없다사내는 그런 자신의...,익명,2020-12-01,00:08,245.0,7.0,0.0,['거사사 너때 때멈 살그... 라고요???? 말이 너무 심하시네요!! 비추드립니다...,"['뭐라는거야ㅋㅋㅋㅋ', '저도 쓰고보니 이상 거울이랑 윤동주 자화상이랑 살짝 섞인...",8.0,0.0,"[(거울, Noun), (속, Noun), (사내, Noun), (서있다, Verb...","[(시, Noun), (써, Verb), (봤는데, Verb)]","[("", ,), (사사, Noun), (때멈, Noun), (살그, Noun), (...",[]
161971,1058729,익게2,민법 한번 공부해 보고싶은데.. 인강 뭐 들어야할지 모르겠어ㅜ,세무사 민법이랑 법무사 민법...,익명,2020-12-01,00:09,221.0,0.0,2.0,"['로스쿨 민법 어때? - 로스쿨다미', '자기계발용으로는 적합한지 모르겠지만ㅋㅋㅋ...","['헤헤 그거 들어볼까?? 혹시 추천 강사님 물어봐도돼?ㅎ', 'ㅋㅋㅋㅋ']",8.0,0.0,"[(세무사, Noun), (민법, Noun), (법무사, Noun), (민법, No...","[(민법, Noun), (한번, Noun), (공부, Noun), (해, Verb)...","[("", ,), (로스쿨, Noun), (민법, Noun), (?, Punctuat...",[]
161972,1058730,익게2,가정내에서도 결국 사회생활하듯이 살아야할까,나는 엄마랑 감정적으로 친해지고 싶지 않아말하는 관점이 너어어어무 다르고 남 험담을...,익명,2020-12-01,00:10,376.0,11.0,0.0,"['솔직히 내가 그 상황이 아니니까 답은 알 수 없지만, 나라면 안 드려 이번에 드...",['진짜 고민이야 ㅠㅠ 정말 오늘 아침도 또 아빠욕하길래 못들어주겟다고 하고 나왔는...,11.0,0.0,"[(엄마, Noun), (랑, Josa), (감정, Noun), (적, Suffix...","[(가정, Noun), (내, Noun), (에서도, Josa), (사회생활, No...","[("", ,), (솔직히, Adjective), (내, Noun), (상황, Nou...",[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77975,1200285,익게2,아버지께서 칼럼 쓰셨는데 댓글 한 번씩만 달아줄 수 있을까,2222408544049우리 아버지께서 이번에 짧은 칼럼처럼 블로그에 쓰시는 거 맡...,익명,2021-06-30,23:43,5802.0,374.0,11.0,"비댓으로 달래 너무 귀엽다, 이 글 조회수 늘리려면 제목 바꾸는거 추천 광고글인줄 ...","헉 피드백 반영했어 너무 고마워, 엇 아마 확인은 못하실 텐데 비댓이든 공개댓이든 ...",75.0,NaN,"[(2222408544049, Number), (우리, Noun), (아버지, No...","[(아버지, Noun), (께서, Josa), (칼럼, Noun), (쓰셨는데, V...","[(비댓, Noun), (으로, Josa), (달래, Noun), (너무, Adve...","[(헉, Adverb), (피드백, Noun), (반영, Noun), (했어, Ve..."
77976,1200287,익게2,프로포폴 하면 기분이 좋음,하는거 신기하네하면 힙해보여서 하는건가,익명,2021-06-30,23:47,373.0,1.0,1.0,썰로 들은 거라 정확하지는 않는데 지방흡입할 때 고통 줄이려고 프로포폴 투약하다가 ...,,5.0,NaN,"[(하는거, Verb), (신기하네하면, Adjective), (힙, Noun), ...","[(프로포폴, Noun), (하면, Verb), (기분, Noun), (이, Jos...","[(썰로, Verb), (들은, Verb), (거, Noun), (라, Josa),...",[]
77977,1200288,익게2,취업 후기 쓰려는데 안올라간다ㅠ,손다쳐서 폰으로 열심히 적었는데나한테왜 이래ㅠ혹시 뭔가 지켜야 하는 양식이 있나요ㅠ...,익명,2021-06-30,23:49,387.0,3.0,0.0,"중간에 임티 넣었었어, 임티, 1 이모티콘 빼기2 사진 많으면 업로드 될 동안 조금...",이건 되네,7.0,NaN,"[(손, Noun), (다쳐서, Verb), (폰, Noun), (으로, Josa)...","[(취업, Noun), (후기, Noun), (쓰려는데, Verb), (안, Ver...","[(중간, Noun), (에, Josa), (임티, Noun), (넣었었어, Ver...","[(이건, Noun), (되네, Verb)]"
77978,1200291,익게2,간떨어지는동거 보는 사람,아 이번주 기대했는데 예고편...,익명,2021-06-30,23:55,138.0,6.0,0.0,"너무 오글거리는데 장기용때매 봄 ㅎㅎ, 난 계선우땜에 봐ㅠㅠ, 나도 계선우 때문에 ...",,3.0,NaN,"[(아, Exclamation), (이번, Noun), (주, Noun), (기대했...","[(간, Noun), (떨어지는, Verb), (동거, Noun), (보는, Ver...","[(너무, Adverb), (오글거리는데, Verb), (장기, Noun), (용때...",[]


In [620]:
len(df2)

95166

In [357]:
print(sorted(Points[1],reverse=True))

[6.703297343371216, 4.678358382013887, 2.555023273587537, 1.225809159656065, 1.0049050375669006, 0.8877339435935453, 0.7919453733634467, 0.6383163260482457, 0.6135197393016488, 0.6060421522225551, 0.5788879473158969, 0.57136717368269, 0.5659098384611532, 0.5658716531841714, 0.5613042486816671, 0.5477407448187572, 0.5422536403773736, 0.5406715533311987, 0.5300093857672518, 0.5231311263242127, 0.5176197962013431, 0.5165226228450097, 0.5152824356062762, 0.5115552371169461, 0.5112337185756655, 0.5102956869670241, 0.5045383290835956, 0.5039559719314464, 0.5028046981343652, 0.49378005345945253, 0.49213025197351634, 0.4909003633607463, 0.4900067679089991, 0.4895741119921423, 0.48952867632403996, 0.48737676677958364, 0.48067708815051136, 0.4785136688411017, 0.47740361643270124, 0.47329871926604794, 0.4725548673393558, 0.46916975524787363, 0.46898282276954245, 0.46797384237412865, 0.46585468843260947, 0.46291169392489584, 0.462370597945963, 0.46094788982567714, 0.45981572602622167, 0.4588119764

In [328]:
for i in  BoW_mat[1]:
    print(i)

6.0
8.0
1.0
16.0
24.0
145.0
224.0
15.0
861.0
1890.0
173.0
120.0
4386.0
36.0
51.0
988.0
7.0
529.0
23.0
16.0
37.0
313.0
127.0
116.0
269.0
55.0
240.0
282.0
419.0
907.0
182.0
251.0
241.0
54.0
266.0
0.0
76.0
3.0
74.0
0.0
4.0
0.0
286.0
262.0
9.0
12.0
7.0
496.0
9.0
80.0
12.0
41.0
42.0
190.0
50.0
44.0
31.0
354.0
217.0
654.0
221.0
69.0
503.0
25.0
14.0
36.0
263.0
410.0
0.0
3.0
1390.0
493.0
103.0
767.0
35.0
141.0
99.0
51.0
16.0
3.0
13.0
2.0
24.0
538.0
1805.0
1009.0
117.0
11.0
29.0
118.0
121.0
0.0
68.0
3.0
17.0
1.0
11.0
69.0
934.0
62.0
4.0
14.0
18.0
12.0
48.0
312.0
0.0
15.0
2.0
580.0
2310.0
198.0
40.0
53.0
12.0
253.0
14.0
2.0
237.0
10.0
14.0
33.0
58.0
284.0
74.0
122.0
31.0
217.0
14.0
0.0
49.0
330.0
16.0
83.0
338.0
0.0
170.0
84.0
10.0
188.0
101.0
22.0
308.0
21.0
0.0
1038.0
40.0
6.0
18.0
158.0
3.0
31.0
0.0
13.0
0.0
888.0
177.0
91.0
9.0
1.0
0.0
40.0
5172.0
85.0
56.0
96.0
91.0
3954.0
96.0
4.0
4.0
19.0
2.0
5.0
366.0
14.0
14.0
78.0
61.0
322.0
133.0
55.0
215.0
42.0
292.0
11.0
536.0
16.0
30.0
5.0
81.0
0.0

21.0
6.0
10.0
22.0
0.0
7.0
30.0
82.0
38.0
30.0
34.0
186.0
3.0
31.0
0.0
0.0
3.0
3.0
125.0
1.0
2.0
93.0
2.0
0.0
3.0
15.0
6.0
30.0
31.0
1.0
20.0
101.0
2.0
208.0
134.0
5.0
9.0
0.0
81.0
19.0
12.0
367.0
0.0
85.0
4.0
68.0
2.0
99.0
66.0
12.0
0.0
40.0
24.0
32.0
28.0
27.0
181.0
0.0
99.0
1.0
52.0
1.0
0.0
229.0
42.0
2.0
2.0
6.0
69.0
19.0
31.0
27.0
74.0
124.0
59.0
15.0
66.0
110.0
59.0
17.0
0.0
25.0
3.0
2.0
50.0
0.0
0.0
5.0
9.0
13.0
149.0
29.0
74.0
54.0
98.0
16.0
78.0
42.0
2.0
29.0
122.0
23.0
1.0
22.0
15.0
0.0
14.0
106.0
2.0
27.0
0.0
2.0
1.0
6.0
2.0
24.0
12.0
27.0
53.0
14.0
8.0
22.0
61.0
48.0
15.0
145.0
0.0
100.0
105.0
7.0
104.0
22.0
23.0
12.0
445.0
17.0
3.0
0.0
64.0
14.0
61.0
60.0
4.0
107.0
2.0
9.0
3.0
85.0
9.0
0.0
34.0
14.0
2.0
100.0
1.0
61.0
1.0
1.0
36.0
3.0
27.0
1.0
16.0
28.0
9.0
17.0
3.0
11.0
12.0
48.0
0.0
7.0
17.0
12.0
1.0
16.0
33.0
4.0
6.0
0.0
3.0
9.0
11.0
10.0
46.0
0.0
0.0
77.0
17.0
0.0
92.0
10.0
0.0
3.0
43.0
7.0
40.0
1.0
0.0
15.0
0.0
28.0
8.0
7.0
15.0
9.0
11.0
6.0
1.0
206.0
111.0
4.0
4.0
82

0.0
17.0
14.0
2.0
2.0
0.0
27.0
1.0
7.0
45.0
7.0
0.0
125.0
212.0
3.0
2.0
0.0
3.0
1.0
2.0
40.0
26.0
139.0
0.0
4.0
27.0
91.0
0.0
25.0
17.0
24.0
1.0
3.0
37.0
0.0
45.0
38.0
6.0
286.0
287.0
0.0
2.0
12.0
137.0
6.0
86.0
16.0
59.0
0.0
1.0
0.0
17.0
16.0
13.0
4.0
19.0
15.0
1.0
5.0
2.0
6.0
2.0
24.0
3.0
0.0
58.0
33.0
29.0
1.0
1.0
3.0
30.0
41.0
18.0
28.0
14.0
39.0
19.0
15.0
4.0
10.0
60.0
21.0
0.0
4.0
0.0
8.0
140.0
4.0
0.0
4.0
0.0
13.0
13.0
0.0
0.0
32.0
0.0
1.0
0.0
1.0
31.0
11.0
4.0
0.0
5.0
10.0
8.0
31.0
69.0
15.0
53.0
0.0
2.0
12.0
2.0
10.0
13.0
35.0
19.0
6.0
33.0
0.0
98.0
22.0
27.0
5.0
3.0
18.0
10.0
2.0
17.0
0.0
73.0
17.0
0.0
0.0
7.0
2.0
0.0
0.0
0.0
2.0
0.0
40.0
7.0
79.0
7.0
154.0
5.0
22.0
4.0
13.0
19.0
1.0
3.0
18.0
1.0
8.0
5.0
11.0
0.0
20.0
7.0
0.0
7.0
79.0
3.0
20.0
15.0
6.0
41.0
30.0
39.0
28.0
87.0
39.0
80.0
3.0
0.0
2.0
7.0
26.0
6.0
75.0
12.0
0.0
28.0
45.0
5.0
34.0
0.0
74.0
14.0
1.0
10.0
12.0
32.0
5.0
1.0
55.0
35.0
0.0
0.0
21.0
13.0
7.0
133.0
62.0
16.0
1.0
0.0
1.0
36.0
0.0
0.0
2.0
1.0
33.0
3.0
4.0

19.0
4.0
47.0
3.0
41.0
13.0
0.0
4.0
0.0
3.0
2.0
0.0
0.0
1.0
0.0
1.0
1.0
0.0
11.0
2.0
1.0
0.0
7.0
0.0
1.0
25.0
55.0
14.0
2.0
5.0
4.0
0.0
3.0
2.0
31.0
5.0
19.0
0.0
18.0
1.0
1.0
5.0
3.0
2.0
22.0
1.0
1.0
6.0
0.0
0.0
0.0
0.0
10.0
9.0
0.0
0.0
29.0
10.0
0.0
6.0
0.0
9.0
0.0
38.0
0.0
27.0
0.0
0.0
8.0
0.0
5.0
11.0
5.0
20.0
6.0
4.0
0.0
0.0
4.0
3.0
2.0
1.0
4.0
11.0
5.0
3.0
24.0
17.0
0.0
2.0
0.0
9.0
9.0
11.0
35.0
1.0
1.0
12.0
23.0
15.0
11.0
40.0
23.0
6.0
6.0
3.0
7.0
48.0
7.0
1.0
18.0
1.0
4.0
0.0
3.0
7.0
30.0
62.0
2.0
1.0
4.0
0.0
0.0
0.0
34.0
22.0
3.0
11.0
1.0
2.0
2.0
4.0
0.0
1.0
5.0
13.0
14.0
35.0
0.0
32.0
35.0
3.0
13.0
6.0
22.0
6.0
2.0
2.0
1.0
1.0
5.0
1.0
134.0
2.0
74.0
0.0
7.0
0.0
40.0
0.0
1.0
2.0
40.0
5.0
12.0
4.0
0.0
2.0
9.0
7.0
21.0
5.0
3.0
57.0
2.0
18.0
14.0
4.0
33.0
4.0
5.0
13.0
8.0
11.0
9.0
0.0
7.0
1.0
0.0
7.0
7.0
2.0
3.0
27.0
0.0
7.0
0.0
11.0
4.0
0.0
29.0
0.0
33.0
39.0
60.0
1.0
26.0
0.0
1.0
58.0
23.0
0.0
2.0
0.0
14.0
47.0
33.0
14.0
0.0
49.0
34.0
0.0
4.0
41.0
12.0
7.0
10.0
1.0
11.0
12.0
5.0

10.0
2.0
3.0
25.0
3.0
2.0
1.0
4.0
1.0
1.0
6.0
6.0
3.0
22.0
4.0
6.0
0.0
0.0
15.0
8.0
1.0
0.0
86.0
33.0
0.0
5.0
16.0
10.0
35.0
7.0
32.0
11.0
0.0
1.0
1.0
0.0
107.0
13.0
1.0
18.0
261.0
30.0
0.0
2.0
5.0
0.0
0.0
4.0
25.0
0.0
0.0
0.0
1.0
4.0
2.0
66.0
5.0
23.0
7.0
9.0
2.0
2.0
3.0
19.0
27.0
80.0
38.0
6.0
1.0
4.0
7.0
6.0
2.0
1.0
2.0
0.0
24.0
5.0
2.0
0.0
1.0
10.0
2.0
2.0
8.0
4.0
3.0
6.0
1.0
3.0
4.0
0.0
14.0
1.0
0.0
2.0
1.0
2.0
0.0
4.0
0.0
5.0
2.0
6.0
0.0
3.0
1.0
12.0
0.0
90.0
15.0
1.0
8.0
41.0
2.0
3.0
85.0
24.0
17.0
5.0
30.0
30.0
6.0
26.0
8.0
5.0
10.0
68.0
2.0
0.0
19.0
8.0
6.0
1.0
3.0
4.0
3.0
0.0
179.0
3.0
9.0
15.0
1.0
9.0
16.0
0.0
27.0
7.0
0.0
0.0
17.0
0.0
44.0
29.0
1.0
11.0
3.0
0.0
47.0
11.0
1.0
2.0
0.0
0.0
6.0
0.0
44.0
3.0
14.0
7.0
19.0
49.0
3.0
6.0
4.0
7.0
2.0
0.0
42.0
1.0
1.0
0.0
2.0
47.0
1.0
19.0
20.0
0.0
2.0
6.0
3.0
4.0
15.0
1.0
5.0
8.0
13.0
7.0
27.0
11.0
8.0
1.0
2.0
0.0
0.0
0.0
7.0
20.0
0.0
3.0
0.0
1.0
7.0
4.0
0.0
1.0
19.0
7.0
15.0
22.0
5.0
2.0
0.0
2.0
2.0
0.0
0.0
1.0
0.0
0.0
10.0
1.0
5.0

...

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
